In [26]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date

In [27]:
## I acquire stock list in thailand ###
ex = pd.read_excel('listedCompanies_th_TH.xlsx', sheet_name='listedCompanies_th_TH',skiprows=1)
df = pd.DataFrame(ex['หลักทรัพย์'].astype('str') + '.bk')
df = df.rename(columns={'หลักทรัพย์' : 'ticker'})
df = df.reset_index(drop=True)
df

,ticker
0,24CS.bk
1,2S.bk
2,3BBIF.bk
3,3K-BAT.bk
4,A.bk
...,...
916,YONG.bk
917,YUASA.bk
918,ZAA.bk
919,ZEN.bk


### Acquire informations from yfinance function >> .info

In [28]:
# 1st pulling date for this data #
df['date_pulling'] = date.today()

In [29]:
# 2nd acquiring information from .info #
info_attribute_list = [
    'industry',
    'sector',
    'marketCap',
    'currentPrice',
    'enterpriseValue',
]
# 3rd acquiring information from .quarterly_balance_sheet #
balancesheet_list = [
    'Total Assets',
    'Current Liabilities'
]

# 4th acquiring information from .quarterly_financials #
ttm = 4
financials_list = [
    'EBIT'
]

info_attribute_list_buffer = []
balancesheet_list_buffer = []
financials_list_buffer = []
for i in range(len(df)):
    yfticker = yf.Ticker(df['ticker'][i])

    # 2nd acquiring information from .info #
    info_attribute_list_buffer_r = []
    for j in range(len(info_attribute_list)):
        try:
            info_attribute_list_buffer_r.append(yfticker.info[info_attribute_list[j]])
        except:
            info_attribute_list_buffer_r.append(None)
    info_attribute_list_buffer.append(info_attribute_list_buffer_r)

    # 3rd acquiring information from .balancesheet #
    balancesheet_list_buffer_r = []
    for j in range(len(balancesheet_list)):
        try:
            balancesheet_list_buffer_r.append(yfticker.quarterly_balance_sheet.loc[yfticker.quarterly_balance_sheet.index == balancesheet_list[j]].values[0][0])
        except:
            balancesheet_list_buffer_r.append(None)
    balancesheet_list_buffer.append(balancesheet_list_buffer_r)

    # 4th acquiring information from .quarterly_financials #
    financials_list_buffer_r = []
    for j in range(len(financials_list)):
        try:
            financials_list_buffer_r.append(np.array([yfticker.quarterly_financials.loc[yfticker.quarterly_financials.index == financials_list[j]].values[0][i] for i in range(ttm)]).sum())
        except:
            financials_list_buffer_r.append(None)
    financials_list_buffer.append(financials_list_buffer_r)

df = df.join(pd.DataFrame(info_attribute_list_buffer, columns=info_attribute_list))
df = df.join(pd.DataFrame(balancesheet_list_buffer, columns=balancesheet_list))
df = df.join(pd.DataFrame(financials_list_buffer, columns=financials_list))
df


,ticker,date_pulling,industry,sector,marketCap,currentPrice,enterpriseValue,Total Assets,Current Liabilities,EBIT
0,24CS.bk,2024-08-08,Building Products & Equipment,Industrials,1.883400e+09,4.38,1.917655e+09,7.280455e+08,1.470868e+08,-99415142.0
1,2S.bk,2024-08-08,Steel,Basic Materials,1.550989e+09,2.82,1.292300e+09,2.400035e+09,3.039420e+08,122502000.0
2,3BBIF.bk,2024-08-08,None,None,NaN,5.65,NaN,NaN,NaN,NaN
3,3K-BAT.bk,2024-08-08,Electrical Equipment & Parts,Industrials,4.155200e+09,53.00,3.860087e+09,3.736470e+09,8.851644e+08,205445883.0
4,A.bk,2024-08-08,Real Estate - Development,Real Estate,4.880400e+09,4.98,1.386429e+10,1.318669e+10,2.791347e+09,-155867045.0
...,...,...,...,...,...,...,...,...,...,...
916,YONG.bk,2024-08-08,Building Materials,Basic Materials,9.723999e+08,1.43,1.302733e+09,1.307177e+09,2.661980e+08,170391234.0
917,YUASA.bk,2024-08-08,Auto Parts,Consumer Cyclical,1.162350e+09,10.80,7.642700e+08,1.646425e+09,3.800130e+08,181854120.0
918,ZAA.bk,2024-08-08,Entertainment,Communication Services,8.320768e+08,0.64,5.769555e+08,6.041760e+08,1.868430e+08,-39519945.0
919,ZEN.bk,2024-08-08,Restaurants,Consumer Cyclical,1.774201e+09,5.95,2.678923e+09,3.230292e+09,9.035990e+08,221103000.0


In [30]:
df = df.dropna()
df = df.reset_index(drop=True)
### Next I perform calculations for MF_ROC and MF_EY ###
df['MF_ROC'] = df['EBIT']/(df['Total Assets'] - df['Current Liabilities'])
df['MF_EY'] = df['EBIT']/df['enterpriseValue']
df

,ticker,date_pulling,industry,sector,marketCap,currentPrice,enterpriseValue,Total Assets,Current Liabilities,EBIT,MF_ROC,MF_EY
0,24CS.bk,2024-08-08,Building Products & Equipment,Industrials,1.883400e+09,4.38,1.917655e+09,7.280455e+08,1.470868e+08,-99415142.0,-0.171123,-0.051842
1,2S.bk,2024-08-08,Steel,Basic Materials,1.550989e+09,2.82,1.292300e+09,2.400035e+09,3.039420e+08,122502000.0,0.058443,0.094794
2,3K-BAT.bk,2024-08-08,Electrical Equipment & Parts,Industrials,4.155200e+09,53.00,3.860087e+09,3.736470e+09,8.851644e+08,205445883.0,0.072053,0.053223
3,A.bk,2024-08-08,Real Estate - Development,Real Estate,4.880400e+09,4.98,1.386429e+10,1.318669e+10,2.791347e+09,-155867045.0,-0.014994,-0.011242
4,A5.bk,2024-08-08,Real Estate - Development,Real Estate,3.369080e+09,2.86,4.631671e+09,3.836102e+09,1.114288e+09,691593519.0,0.254093,0.149318
...,...,...,...,...,...,...,...,...,...,...,...,...
792,YONG.bk,2024-08-08,Building Materials,Basic Materials,9.723999e+08,1.43,1.302733e+09,1.307177e+09,2.661980e+08,170391234.0,0.163684,0.130795
793,YUASA.bk,2024-08-08,Auto Parts,Consumer Cyclical,1.162350e+09,10.80,7.642700e+08,1.646425e+09,3.800130e+08,181854120.0,0.143598,0.237945
794,ZAA.bk,2024-08-08,Entertainment,Communication Services,8.320768e+08,0.64,5.769555e+08,6.041760e+08,1.868430e+08,-39519945.0,-0.094696,-0.068497
795,ZEN.bk,2024-08-08,Restaurants,Consumer Cyclical,1.774201e+09,5.95,2.678923e+09,3.230292e+09,9.035990e+08,221103000.0,0.095029,0.082534


In [31]:
### Ranking regarding MFs####

df['Ranking_MF_ROC'] = df['MF_ROC'].rank()
df['Ranking_MF_EY'] = df['MF_EY'].rank()
df['Ranking_MF'] = df['Ranking_MF_ROC'] + df['Ranking_MF_EY']

df.loc[df['Ranking_MF'] == df['Ranking_MF'].max()]

,ticker,date_pulling,industry,sector,marketCap,currentPrice,enterpriseValue,Total Assets,Current Liabilities,EBIT,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
304,JAS.bk,2024-08-08,Telecom Services,Communication Services,2.205691e+10,2.66,1.967599e+10,1.799440e+10,4.457690e+09,2.261631e+10,1.670739,1.149437,796.0,797.0,1593.0


In [36]:
df_sorted = df.sort_values(by=['Ranking_MF'],ascending=False)
df_sorted = df_sorted.reset_index(drop=True)
df_sorted[:50]

,ticker,date_pulling,industry,sector,marketCap,currentPrice,enterpriseValue,Total Assets,Current Liabilities,EBIT,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
0,JAS.bk,2024-08-08,Telecom Services,Communication Services,2.205691e+10,2.66,1.967599e+10,1.799440e+10,4.457690e+09,2.261631e+10,1.670739,1.149437,796.0,797.0,1593.0
1,LANNA.bk,2024-08-08,Thermal Coal,Energy,7.402500e+09,14.10,7.739623e+09,1.622346e+10,3.418211e+09,4.825900e+09,0.376869,0.623532,789.0,790.0,1579.0
2,Q-CON.bk,2024-08-08,Building Products & Equipment,Industrials,3.980000e+09,9.95,2.145099e+09,3.412410e+09,8.513570e+08,1.012475e+09,0.395336,0.471995,793.0,786.0,1579.0
3,PIN.bk,2024-08-08,Real Estate—Development,Real Estate,4.964800e+09,4.28,5.895888e+09,6.263542e+09,6.600270e+08,1.774394e+09,0.316657,0.300954,779.0,766.0,1545.0
4,BTC.bk,2024-08-08,Financial Conglomerates,Financial Services,5.897472e+09,0.48,3.441970e+09,4.202732e+09,5.192318e+08,1.046087e+09,0.283993,0.303921,774.0,768.0,1542.0
5,SRICHA.bk,2024-08-08,Engineering & Construction,Industrials,2.308599e+09,7.45,1.368630e+09,2.911026e+09,6.521662e+08,5.931280e+08,0.262579,0.433374,759.0,782.0,1541.0
6,JCK.bk,2024-08-08,Real Estate—Development,Real Estate,1.254908e+09,0.33,6.154825e+09,9.097491e+09,3.840008e+09,1.562178e+09,0.297134,0.253814,777.0,751.0,1528.0
7,SUSCO.bk,2024-08-08,Oil & Gas Refining & Marketing,Energy,3.280000e+09,3.28,5.377212e+09,1.002048e+10,3.305554e+09,1.593605e+09,0.237323,0.296363,753.0,764.0,1517.0
8,CAZ.bk,2024-08-08,Engineering & Construction,Industrials,7.879200e+08,2.68,1.027790e+09,2.631247e+09,1.488775e+09,2.827815e+08,0.247517,0.275135,756.0,755.0,1511.0
9,PRI.bk,2024-08-08,Real Estate Services,Real Estate,2.272000e+09,7.10,1.648800e+09,2.572253e+09,6.192820e+08,4.616862e+08,0.236402,0.280013,752.0,759.0,1511.0
